# General info and setup


use this jupyter notebook to load taunus-medusa data and use baseline filter to get estimates for summer 2025

In [ ]:
# Automatically reload modules when they change
%load_ext autoreload

# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

In [ ]:
# Package and library imports
import xarray as xr
import numpy as np
from pathlib import Path
import pandas as pd

%matplotlib widget
from matplotlib import pyplot as plt
import seaborn as sns

import IAU_baseline.baseline as base
from vsls_2025.plotting import run_interactive  # in order to catch plots from subroutines and display them interactively in notebook

In [ ]:
# Set up file paths
data_dir = Path("../data/raw/")
fname_ch2cl2 = data_dir / "AGAGE-GCMS-Medusa_TOB_ch2cl2.nc"
fname_ch3br = data_dir / "AGAGE-GCMS-Medusa_TOB_ch3br.nc"
fname_ch3cl = data_dir / "AGAGE-GCMS-Medusa_TOB_ch3cl.nc"
fname_chcl3 = data_dir / "AGAGE-GCMS-Medusa_TOB_chcl3.nc"
fname_pce = data_dir / "AGAGE-GCMS-Medusa_TOB_pce.nc"
fname_cfc11 = data_dir / "AGAGE-GCMS-Medusa_TOB_cfc-11.nc"
fname_cfc12 = data_dir / "AGAGE-GCMS-Medusa_TOB_cfc-12.nc"
fname_h1211 = data_dir / "AGAGE-GCMS-Medusa_TOB_h-1211.nc"
fname_hcfc22 = data_dir / "AGAGE-GCMS-Medusa_TOB_hcfc-22.nc"
fname_hfc32 = data_dir / "AGAGE-GCMS-Medusa_TOB_hfc-32.nc"
fname_hfc125 = data_dir / "AGAGE-GCMS-Medusa_TOB_hfc-125.nc"
fname_sf6 = data_dir / "AGAGE-GCMS-Medusa_TOB_sf6.nc"

# only use data from 2023-10-10 onwards:
data_start_time = pd.Timestamp("2023-10-10")

# CH2Cl2


This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


<div style="text-align: center;">
  <a href="../external_plots/Screenshot%202026-02-02%20134736.png" target="_blank">
    <img src="../external_plots/Screenshot%202026-02-02%20134736.png" width="50%">
  </a>
</div>

## show the data

In [ ]:
ch2cl2 = xr.open_dataset(fname_ch2cl2)
fig, ax = plt.subplots()
ax.scatter(ch2cl2.time, ch2cl2.mf)
plt.show()

## Apply baseline filter to get estimates for summer 2025

In [ ]:
# preprocess data to get it into a format that can be used for the baseline filter
df = ch2cl2.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

# Filter data: only use data from 10.10.2023 onwards
df = df[df['datetime'] >= data_start_time].reset_index(drop=True)

# Apply baseline filter to get estimates for summer 2025
flag, residual, warning, popt1, baseline = run_interactive(base.find_base, func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)

# Generate filter flags for easier plotting, "pollution event" vs "baseline"
classification = ["pollution event" if x > 0 else "baseline" for x in flag]
df["Classification"] = classification

In [ ]:
# show data with final classification
# maybe this code cell needs to be run twice
fig, ax = plt.subplots()
sns.scatterplot(x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
plt.show()

## plot the fitted curve using the fit function and the fitted parameters 

In [ ]:
# convert time axis to normalized (shifted) numerical values 
df["fractional_year"] = base.datetime_to_fractionalyear(df["datetime"])  # fractional year values 
df["fractional_year_yy"] = df["fractional_year"] - np.floor(df.loc[df.index[0],"fractional_year"])  # shift to start at 0 to match fit function
y_fit = base.fct.higher(df["fractional_year_yy"], *popt1)  # calculate fitted curve

# plot data and fitted curve together, using the original datetime values for the x-axis and the fitted curve values for the y-axis
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
sns.scatterplot(ax=ax, x=df["datetime"], y=y_fit ,edgecolor=None, linewidth=0, color="k", s=15)

# uncertainty band
uncertainty = np.nanmean(residual)
ax.fill_between(df["datetime"], y_fit - uncertainty, y_fit + uncertainty, 
                color='black', alpha=0.3, linewidth=0)

ax.set_ylim(60, 115)
plt.show()

## Create dataframe for fitting parameters output

In [ ]:
popt1
np.nanmean(residual)

In [ ]:
parameters_df = pd.DataFrame({
    "Parameter": ["a", "b", "c", "d", "e", "f", "g", "uncertainty"],
    "CH2Cl2": list(popt1) + [np.nanmean(residual)]
})
parameters_df

# CHCl3


This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


<div style="text-align: center;">
  <a href="../external_plots\Screenshot 2026-02-16 152743.png" target="_blank">
    <img src="../external_plots\Screenshot 2026-02-16 152743.png" width="50%">
  </a>
</div>

## show the data

In [ ]:
chcl3 = xr.open_dataset(fname_chcl3)
fig, ax = plt.subplots()
ax.scatter(chcl3.time, chcl3.mf)
plt.show()

## Apply baseline filter to get estimates for summer 2025

In [ ]:
# preprocess data to get it into a format that can be used for the baseline filter
df = chcl3.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

# Filter data: only use data from 10.10.2023 onwards
df = df[df['datetime'] >= data_start_time].reset_index(drop=True)

# Apply baseline filter to get estimates for summer 2025
flag, residual, warning, popt1, baseline = run_interactive(base.find_base, func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)

# Generate filter flags for easier plotting, "pollution event" vs "baseline"
classification = ["pollution event" if x > 0 else "baseline" for x in flag]
df["Classification"] = classification

In [ ]:
# show data with final classification
# maybe this code cell needs to be run twice
fig, ax = plt.subplots()
sns.scatterplot(x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
plt.show()

## plot the fitted curve using the fit function and the fitted parameters 

In [ ]:
# convert time axis to normalized (shifted) numerical values 
df["fractional_year"] = base.datetime_to_fractionalyear(df["datetime"])  # fractional year values 
df["fractional_year_yy"] = df["fractional_year"] - np.floor(df.loc[df.index[0],"fractional_year"])  # shift to start at 0 to match fit function
y_fit = base.fct.higher(df["fractional_year_yy"], *popt1)  # calculate fitted curve

# plot data and fitted curve together, using the original datetime values for the x-axis and the fitted curve values for the y-axis
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
sns.scatterplot(ax=ax, x=df["datetime"], y=y_fit ,edgecolor=None, linewidth=0, color="k", s=15)

# uncertainty band
uncertainty = np.nanmean(residual)
ax.fill_between(df["datetime"], y_fit - uncertainty, y_fit + uncertainty, 
                color='black', alpha=0.3, linewidth=0)

ax.set_ylim(8.5, 17)
plt.show()

## Append dataframe for fitting parameters output

In [ ]:
popt1
np.nanmean(residual)

In [ ]:
parameters_df["CHCl3"] = list(popt1) + [np.nanmean(residual)]
parameters_df

# C2Cl4 (PCE)


This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


<div style="text-align: center;">
  <a href="../external_plots\Screenshot 2026-02-16 150552.png" target="_blank">
    <img src="../external_plots\Screenshot 2026-02-16 150552.png" width="50%">
  </a>
</div>

## show the data

In [ ]:
pce = xr.open_dataset(fname_pce)
fig, ax = plt.subplots()
ax.scatter(pce.time, pce.mf)
plt.show()

## Apply baseline filter to get estimates for summer 2025

In [ ]:
# preprocess data to get it into a format that can be used for the baseline filter
df = pce.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

# Filter data: only use data from 10.10.2023 onwards
df = df[df['datetime'] >= data_start_time].reset_index(drop=True)

# Apply baseline filter to get estimates for summer 2025
flag, residual, warning, popt1, baseline = run_interactive(base.find_base, func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)

# Generate filter flags for easier plotting, "pollution event" vs "baseline"
classification = ["pollution event" if x > 0 else "baseline" for x in flag]
df["Classification"] = classification

In [ ]:
# show data with final classification
# maybe this code cell needs to be run twice
fig, ax = plt.subplots()
sns.scatterplot(x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
plt.show()

## plot the fitted curve using the fit function and the fitted parameters 

In [ ]:
# convert time axis to normalized (shifted) numerical values 
df["fractional_year"] = base.datetime_to_fractionalyear(df["datetime"])  # fractional year values 
df["fractional_year_yy"] = df["fractional_year"] - np.floor(df.loc[df.index[0],"fractional_year"])  # shift to start at 0 to match fit function
y_fit = base.fct.higher(df["fractional_year_yy"], *popt1)  # calculate fitted curve

# plot data and fitted curve together, using the original datetime values for the x-axis and the fitted curve values for the y-axis
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
sns.scatterplot(ax=ax, x=df["datetime"], y=y_fit ,edgecolor=None, linewidth=0, color="k", s=15)

# uncertainty band
uncertainty = np.nanmean(residual)
ax.fill_between(df["datetime"], y_fit - uncertainty, y_fit + uncertainty, 
                color='black', alpha=0.3, linewidth=0)
ax.set_ylim(1, 6)

plt.show()

## Append dataframe for fitting parameters output

In [ ]:
popt1
np.nanmean(residual)

In [ ]:
parameters_df["C2Cl4"] = list(popt1) + [np.nanmean(residual)]
parameters_df

# CH3Cl


This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


<div style="text-align: center;">
  <a href="../external_plots\Screenshot 2026-02-16 144028.png" target="_blank">
    <img src="../external_plots\Screenshot 2026-02-16 144028.png" width="50%">
  </a>
</div>

## show the data

In [ ]:
ch3cl = xr.open_dataset(fname_ch3cl)
fig, ax = plt.subplots()
ax.scatter(ch3cl.time, ch3cl.mf)
plt.show()

## Apply baseline filter to get estimates for summer 2025

In [ ]:
# preprocess data to get it into a format that can be used for the baseline filter
df = ch3cl.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

# Filter data: only use data from 10.10.2023 onwards
df = df[df['datetime'] >= data_start_time].reset_index(drop=True)

# Apply baseline filter to get estimates for summer 2025
flag, residual, warning, popt1, baseline = run_interactive(base.find_base, func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)

# Generate filter flags for easier plotting, "pollution event" vs "baseline"
classification = ["pollution event" if x > 0 else "baseline" for x in flag]
df["Classification"] = classification

In [ ]:
# show data with final classification
# maybe this code cell needs to be run twice
fig, ax = plt.subplots()
sns.scatterplot(x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
plt.show()

## plot the fitted curve using the fit function and the fitted parameters 

In [ ]:
# convert time axis to normalized (shifted) numerical values 
df["fractional_year"] = base.datetime_to_fractionalyear(df["datetime"])  # fractional year values 
df["fractional_year_yy"] = df["fractional_year"] - np.floor(df.loc[df.index[0],"fractional_year"])  # shift to start at 0 to match fit function
y_fit = base.fct.higher(df["fractional_year_yy"], *popt1)  # calculate fitted curve

# plot data and fitted curve together, using the original datetime values for the x-axis and the fitted curve values for the y-axis
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
sns.scatterplot(ax=ax, x=df["datetime"], y=y_fit ,edgecolor=None, linewidth=0, color="k", s=15)

# uncertainty band
uncertainty = np.nanmean(residual)
ax.fill_between(df["datetime"], y_fit - uncertainty, y_fit + uncertainty, 
                color='black', alpha=0.3, linewidth=0)
ax.set_ylim(460, 630)

plt.show()

## Append dataframe for fitting parameters output

In [ ]:
popt1
np.nanmean(residual)

In [ ]:
parameters_df["CH3Cl"] = list(popt1) + [np.nanmean(residual)]
parameters_df

# CH3Br (cannot be used due to contamination)


This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


<div style="text-align: center;">
  <a href="../external_plots\Screenshot 2026-02-16 170843.png" target="_blank">
    <img src="../external_plots\Screenshot 2026-02-16 170843.png" width="50%">
  </a>
</div>

# CFC-11


This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


<div style="text-align: center;">
  <a href="../external_plots\Screenshot 2026-02-16 171848.png" target="_blank">
    <img src="../external_plots\Screenshot 2026-02-16 171848.png" width="50%">
  </a>
</div>

## show the data

In [ ]:
cfc11 = xr.open_dataset(fname_cfc11)
fig, ax = plt.subplots()
ax.scatter(cfc11.time, cfc11.mf)
plt.show()

## Apply baseline filter to get estimates for summer 2025

In [ ]:
# preprocess data to get it into a format that can be used for the baseline filter
df = cfc11.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

# Filter data: only use data from 10.10.2023 onwards
df = df[df['datetime'] >= data_start_time].reset_index(drop=True)

# Apply baseline filter to get estimates for summer 2025
flag, residual, warning, popt1, baseline = run_interactive(base.find_base, func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)

# Generate filter flags for easier plotting, "pollution event" vs "baseline"
classification = ["pollution event" if x > 0 else "baseline" for x in flag]
df["Classification"] = classification

In [ ]:
# show data with final classification
# maybe this code cell needs to be run twice
fig, ax = plt.subplots()
sns.scatterplot(x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
plt.show()

## plot the fitted curve using the fit function and the fitted parameters 

In [ ]:
# convert time axis to normalized (shifted) numerical values 
df["fractional_year"] = base.datetime_to_fractionalyear(df["datetime"])  # fractional year values 
df["fractional_year_yy"] = df["fractional_year"] - np.floor(df.loc[df.index[0],"fractional_year"])  # shift to start at 0 to match fit function
y_fit = base.fct.higher(df["fractional_year_yy"], *popt1)  # calculate fitted curve

# plot data and fitted curve together, using the original datetime values for the x-axis and the fitted curve values for the y-axis
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
sns.scatterplot(ax=ax, x=df["datetime"], y=y_fit ,edgecolor=None, linewidth=0, color="k", s=15)

# uncertainty band
uncertainty = np.nanmean(residual)
ax.fill_between(df["datetime"], y_fit - uncertainty, y_fit + uncertainty, 
                color='black', alpha=0.3, linewidth=0)
ax.set_ylim(210, 220)

plt.show()

## Append dataframe for fitting parameters output

In [ ]:
popt1
np.nanmean(residual)

In [ ]:
parameters_df["CFC-11"] = list(popt1) + [np.nanmean(residual)]
parameters_df

# HCFC-22


This figure shows measurements from 2024 to early 2026, also displaying elevated lab air measurements from mid-2025.


<div style="text-align: center;">
  <a href="../external_plots\Screenshot 2026-02-16 172009.png" target="_blank">
    <img src="../external_plots\Screenshot 2026-02-16 172009.png" width="50%">
  </a>
</div>

## show the data

In [ ]:
hcfc22 = xr.open_dataset(fname_hcfc22)
fig, ax = plt.subplots()
ax.scatter(hcfc22.time, hcfc22.mf)
plt.show()

## Apply baseline filter to get estimates for summer 2025

In [ ]:
# preprocess data to get it into a format that can be used for the baseline filter
df = hcfc22.to_dataframe().reset_index()
df['datetime'] = pd.to_datetime(df['time'], unit='s', origin='unix')

# Filter data: only use data from 10.10.2023 onwards
df = df[df['datetime'] >= data_start_time].reset_index(drop=True)

# Apply baseline filter to get estimates for summer 2025
flag, residual, warning, popt1, baseline = run_interactive(base.find_base, func=base.fct.higher, x_val = df["datetime"], y_val = df["mf"], y_err = df["mf_repeatability"] , flag= None,
                        direction='p', verbose=True, plot= True, ctrl_plots=False, limit= 0.1,
                        stop_rel=True, 
                        post_prc_filter=True)

# Generate filter flags for easier plotting, "pollution event" vs "baseline"
classification = ["pollution event" if x > 0 else "baseline" for x in flag]
df["Classification"] = classification

In [ ]:
# show data with final classification
# maybe this code cell needs to be run twice
fig, ax = plt.subplots()
sns.scatterplot(x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
plt.show()

## plot the fitted curve using the fit function and the fitted parameters 

In [ ]:
# convert time axis to normalized (shifted) numerical values 
df["fractional_year"] = base.datetime_to_fractionalyear(df["datetime"])  # fractional year values 
df["fractional_year_yy"] = df["fractional_year"] - np.floor(df.loc[df.index[0],"fractional_year"])  # shift to start at 0 to match fit function
y_fit = base.fct.higher(df["fractional_year_yy"], *popt1)  # calculate fitted curve

# plot data and fitted curve together, using the original datetime values for the x-axis and the fitted curve values for the y-axis
fig, ax = plt.subplots()
sns.scatterplot(ax=ax, x=df["datetime"], y=df["mf"], hue=df["Classification"],edgecolor=None, linewidth=0)
sns.scatterplot(ax=ax, x=df["datetime"], y=y_fit ,edgecolor=None, linewidth=0, color="k", s=15)

# uncertainty band
uncertainty = np.nanmean(residual)
ax.fill_between(df["datetime"], y_fit - uncertainty, y_fit + uncertainty, 
                color='black', alpha=0.3, linewidth=0)
ax.set_ylim(244, 262)

plt.show()

## Append dataframe for fitting parameters output

In [ ]:
popt1
np.nanmean(residual)

In [ ]:
parameters_df["HCFC-22"] = list(popt1) + [np.nanmean(residual)]
parameters_df